In [1]:
from litellm import completion
from typing import Optional
import litellm
from dotenv import load_dotenv
import rootutils

rootutils.setup_root(".", indicator=".project-root", pythonpath=True)

/Users/shreyasv/miniforge3/envs/retrollm/lib/python3.10/site-packages/pydantic/_internal/_config.py:345: UserWarning: Valid config keys have changed in V2:
* 'fields' has been removed
  warnings.warn(message, UserWarning)


PosixPath('/Users/shreyasv/Desktop/research/deepchem/retrosynthesis/prod')

# Testing

In [12]:
from src.variables import SYS_PROMPT, USER_PROMPT

In [13]:
load_dotenv()
litellm.success_callback = ["langfuse"]
litellm.drop_params = True

metadata = {
    "generation_name": "sv_testing",  # set langfuse generation name
    "project": "Retrosynthesis",  # set langfuse project name
    "version": "0.0.2",  # set langfuse version
    "trace_name": "sv_testing",  # set langfuse Trace Name
    "trace_user_id": "sv",  # set langfuse Trace User ID
    "session_id": "session-1",  # set langfuse Session ID
}


def call_LLM(molecule: str,
             LLM: str = "claude-3-opus-20240229",
             temperature: float = 0.0,
             messages: Optional[list[dict]] = None):
    """Calls the LLM model to predict the next step"""

    # logger.info(f"Calling {LLM} with molecule: {molecule}")
    if messages is None:
        messages = [{
            "role": "system",
            "content": SYS_PROMPT
        }, {
            "role": "user",
            "content": USER_PROMPT.replace('{target_smiles}', molecule)
        }]

    try:
        response = completion(model=LLM,
                              messages=messages,
                              max_completion_tokens=4096,
                              temperature=temperature,
                              seed=42,
                              top_p=0.9,
                              metadata=metadata)
        res_text = response.choices[0].message.content
    except Exception as e:
        # logger.info(f"Error in calling {LLM}: {e}")
        response = completion(model=LLM,
                              messages=messages,
                              max_completion_tokens=4096,
                              temperature=temperature,
                              seed=42,
                              top_p=0.9)
        res_text = response.choices[0].message.content

    # logger.info(f"Response from {LLM}: {response}")
    return res_text


res = call_LLM("CC(=O)CCC")

In [14]:
# extract the content within <cot> </cot> tags as thinking content
thinking_content = res[res.find("<cot>\n") + 6:res.find("</cot>")]
# split the thinking content into individual steps based on the <thinking> </thinking> tags
thinking_steps = thinking_content.split("<thinking>\n")[1:]
thinking_steps = [step[:step.find("</thinking>")] for step in thinking_steps]
print(thinking_content)
print("-------------------")
print(thinking_steps)
print("-------------------")
# extract the content within <json> </json> tags as json content
json_content = res[res.find("<json>\n") + 7:res.find("</json>")]
print(json_content)

<thinking>
The target molecule CC(=O)CCC is a ketone with 5 carbon atoms. Possible retrosynthetic disconnections to consider:

1. Disconnecting the C-C bond adjacent to the ketone, which could arise from an aldol condensation reaction between an enolate and an aldehyde.

2. Disconnecting the C-C bond on the other side of the ketone, which could come from a Grignard addition to a nitrile followed by hydrolysis.

3. Treating the ketone as an electrophile in a Grignard addition reaction with an alkyl halide.

4. Oxidizing a secondary alcohol to form this ketone.
</thinking>

<thinking>
For the aldol condensation approach, we would need an enolate precursor like a ketone or ester with 3 carbons (e.g. acetone or methyl acetate) and an aldehyde with 2 carbons like acetaldehyde. The enolate would add to the aldehyde, followed by dehydration to give the target ketone.
</thinking>

<thinking>
For the Grignard addition to a nitrile, the precursors would be acetonitrile and ethyl magnesium bromid

In [1]:
from litellm import completion
from typing import Optional
import litellm
from dotenv import load_dotenv
import rootutils

rootutils.setup_root(".", indicator=".project-root", pythonpath=True)
from src.metadata import reagent_agent, conditions_agent, literature_agent

In [2]:
from src.cache import clear_cache

clear_cache()

In [3]:
# read the json content
import json

with open(
        "/Users/shreyasv/Desktop/research/deepchem/retrosynthesis/prod/results/mols_small/Zonisamide.json"
) as f:
    output_data = json.load(f)

# output_data = {
#     "dependencies": {
#         "1": ["2"],
#         "2": ["3", "4"],
#         "3": [],
#         "4": []
#     },
#     "steps": [{
#         "step":
#         "1",
#         "reactants": [{
#             "smiles": "COc1ccc(CN(Cc2ccc(OC)cc2)S(=O)(=O)Cc2noc3ccccc23)cc1",
#             "reactant_metadata": {
#                 "name": "",
#                 "chemical_formula": "C24H24N2O5S",
#                 "mass": 452.14059286799994
#             }
#         }],
#         "reagents": [{
#             "smiles": "O=S(=O)(Cl)Cc1noc2ccccc12",
#             "product_metadata": {
#                 "name": "",
#                 "chemical_formula": "C8H6ClNO3S",
#                 "mass": 230.975691732
#             }
#         }],
#         "products": [{
#             "smiles": "NS(=O)(=O)Cc1noc2ccccc12",
#             "product_metadata": {
#                 "name": "",
#                 "chemical_formula": "C8H8N2O3S",
#                 "mass": 212.025563116
#             }
#         }],
#         "conditions": {
#             "temperature":
#             "Room temperature (approximately 20-25\u00b0C)",
#             "pressure":
#             "The reaction is carried out at atmospheric pressure, as no special pressure conditions are typically required for this type of transformation.",
#             "solvent":
#             "This reaction does not require a solvent as the product and reactant are the same molecule. No actual chemical transformation is taking place.",
#             "time":
#             "As no reaction is occurring, the concept of reaction time is not applicable in this case."
#         },
#         "reactionmetrics": [{
#             "scalabilityindex": "8",
#             "confidenceestimate": 0.99,
#             "closestliterature": ""
#         }]
#     }, {
#         "step":
#         "2",
#         "reactants": [{
#             "smiles": "O=S(=O)(Cl)Cc1noc2ccccc12",
#             "reactant_metadata": {
#                 "name": "",
#                 "chemical_formula": "C8H6ClNO3S",
#                 "mass": 230.975691732
#             }
#         }, {
#             "smiles": "COc1ccc(CNCc2ccc(OC)cc2)cc1",
#             "reactant_metadata": {
#                 "name": "",
#                 "chemical_formula": "C16H19NO2",
#                 "mass": 257.141578848
#             }
#         }],
#         "reagents": [{
#             "smiles": "Cc1ccccc1N",
#             "product_metadata": {
#                 "name": "",
#                 "chemical_formula": "C7H9N",
#                 "mass": 107.073499288
#             }
#         }, {
#             "smiles": "Cl[Mg]I",
#             "product_metadata": {
#                 "name": "",
#                 "chemical_formula": "ClIMg",
#                 "mass": 185.85836738
#             }
#         }, {
#             "smiles": "C1CCOC1",
#             "product_metadata": {
#                 "name": "",
#                 "chemical_formula": "C4H8O",
#                 "mass": 72.057514876
#             }
#         }],
#         "products": [{
#             "smiles": "COc1ccc(CN(Cc2ccc(OC)cc2)S(=O)(=O)Cc2noc3ccccc23)cc1",
#             "product_metadata": {
#                 "name": "",
#                 "chemical_formula": "C24H24N2O5S",
#                 "mass": 452.14059286799994
#             }
#         }],
#         "conditions": {
#             "temperature":
#             "Room temperature (20-25 \u00b0C)",
#             "pressure":
#             "The reaction likely proceeds at atmospheric pressure, as no special pressure conditions are typically required for sulfonylation reactions of this type.",
#             "solvent":
#             "A polar aprotic solvent such as dichloromethane (DCM), acetonitrile (MeCN), or N,N-dimethylformamide (DMF)",
#             "time":
#             "1-24 hours, depending on the specific reactivity of the substrates and other reaction conditions"
#         },
#         "reactionmetrics": [{
#             "scalabilityindex": "8",
#             "confidenceestimate": 0.86,
#             "closestliterature": ""
#         }]
#     }, {
#         "step":
#         "3",
#         "reactants": [{
#             "smiles": "O=P(Cl)(Cl)Cl",
#             "reactant_metadata": {
#                 "name": "",
#                 "chemical_formula": "Cl3OP",
#                 "mass": 151.87523428999998
#             }
#         }, {
#             "smiles": "O=S(=O)(O)Cc1noc2ccccc12",
#             "reactant_metadata": {
#                 "name": "",
#                 "chemical_formula": "C8H7NO4S",
#                 "mass": 213.009578704
#             }
#         }],
#         "reagents": [{
#             "smiles": "C1CCOC1",
#             "product_metadata": {
#                 "name": "",
#                 "chemical_formula": "C4H8O",
#                 "mass": 72.057514876
#             }
#         }],
#         "products": [{
#             "smiles": "O=S(=O)(Cl)Cc1noc2ccccc12",
#             "product_metadata": {
#                 "name": "",
#                 "chemical_formula": "C8H6ClNO3S",
#                 "mass": 230.975691732
#             }
#         }],
#         "conditions": {
#             "temperature": "0-25 \u00b0C",
#             "pressure": "1 atm",
#             "solvent":
#             "Dichloromethane (CH2Cl2) or other non-polar aprotic solvent",
#             "time": "1-4 hours"
#         },
#         "reactionmetrics": [{
#             "scalabilityindex": "9",
#             "confidenceestimate": 0.93,
#             "closestliterature": ""
#         }]
#     }, {
#         "step":
#         "4",
#         "reactants": [{
#             "smiles": "COc1ccc(C=O)cc1",
#             "reactant_metadata": {
#                 "name": "",
#                 "chemical_formula": "C8H8O2",
#                 "mass": 136.052429496
#             }
#         }, {
#             "smiles": "COc1ccc(CN)cc1",
#             "reactant_metadata": {
#                 "name": "",
#                 "chemical_formula": "C8H11NO",
#                 "mass": 137.084063972
#             }
#         }],
#         "reagents": [{
#             "smiles": "[H][O][H]",
#             "product_metadata": {
#                 "name": "",
#                 "chemical_formula": "H2O",
#                 "mass": 18.010564684
#             }
#         }, {
#             "smiles": "[H+]",
#             "product_metadata": {
#                 "name": "",
#                 "chemical_formula": "H+",
#                 "mass": 1.00727645209
#             }
#         }],
#         "products": [{
#             "smiles": "COc1ccc(CNCc2ccc(OC)cc2)cc1",
#             "product_metadata": {
#                 "name": "",
#                 "chemical_formula": "C16H19NO2",
#                 "mass": 257.141578848
#             }
#         }],
#         "conditions": {
#             "temperature": "Room temperature (20-25 \u00b0C)",
#             "pressure": "Atmospheric pressure",
#             "solvent": "Ethanol or methanol",
#             "time": "2-4 hours"
#         },
#         "reactionmetrics": [{
#             "scalabilityindex": "9",
#             "confidenceestimate": 0.88,
#             "closestliterature": ""
#         }]
#     }]
# }

In [4]:
for idx, step in enumerate(output_data['steps']):

    status, reagents = reagent_agent(step['reactants'],
                                     step['products'],
                                     LLM="o1-preview-2024-09-12")
    output_data['steps'][idx]['reagents'].extend(reagents)

    status, conditions = conditions_agent(step['reactants'], step['products'],
                                          step['reagents'])
    output_data['steps'][idx]['conditions'] = conditions

    status, literature = literature_agent(step['reactants'], step['products'],
                                          step['reagents'], step['conditions'])
    output_data['steps'][idx]['reactionmetrics'][0][
        'closestliterature'] = literature
    # print(output_data['steps'][idx])
    # break


LookupError: <ContextVar name='logger' at 0x176672570>

In [5]:
print(json.dumps(output_data))

{"dependencies": {"1": ["2"], "2": ["3", "4"], "3": [], "4": []}, "steps": [{"step": "1", "reactants": [{"smiles": "COc1ccc(CN(Cc2ccc(OC)cc2)S(=O)(=O)Cc2noc3ccccc23)cc1", "reactant_metadata": {"name": "", "chemical_formula": "C24H24N2O5S", "mass": 452.14059286799994}}], "reagents": [{"smiles": "O=S(=O)(O)O", "product_metadata": {"name": "", "chemical_formula": "H2O4S", "mass": 97.96737954400001}}, {"smiles": "C1CCOC1", "product_metadata": {"name": "", "chemical_formula": "C4H8O", "mass": 72.057514876}}], "products": [{"smiles": "NS(=O)(=O)Cc1noc2ccccc12", "product_metadata": {"name": "", "chemical_formula": "C8H8N2O3S", "mass": 212.025563116}}], "conditions": {"temperature": "Reflux", "pressure": "Ambient pressure", "solvent": "1,4-Dioxane", "time": "2-4 hours"}, "reactionmetrics": [{"scalabilityindex": "8", "confidenceestimate": 0.99, "closestliterature": "Sulfonylation of amines with sulfonyl chlorides"}]}, {"step": "2", "reactants": [{"smiles": "O=S(=O)(Cl)Cc1noc2ccccc12", "reactant

In [ ]:
out = {
    "dependencies": {
        "1": ["2"],
        "2": ["3", "4"],
        "3": [],
        "4": []
    },
    "steps": [{
        "step":
        "1",
        "reactants": [{
            "smiles": "COc1ccc(CN(Cc2ccc(OC)cc2)S(=O)(=O)Cc2noc3ccccc23)cc1",
            "reactant_metadata": {
                "name": "",
                "chemical_formula": "C24H24N2O5S",
                "mass": 452.14059286799994
            }
        }],
        "reagents": [{
            "smiles": "O=S(=O)(O)O",
            "product_metadata": {
                "name": "",
                "chemical_formula": "H2O4S",
                "mass": 97.96737954400001
            }
        }, {
            "smiles": "C1CCOC1",
            "product_metadata": {
                "name": "",
                "chemical_formula": "C4H8O",
                "mass": 72.057514876
            }
        }],
        "products": [{
            "smiles": "NS(=O)(=O)Cc1noc2ccccc12",
            "product_metadata": {
                "name": "",
                "chemical_formula": "C8H8N2O3S",
                "mass": 212.025563116
            }
        }],
        "conditions": {
            "temperature": "Reflux",
            "pressure": "Ambient pressure",
            "solvent": "1,4-Dioxane",
            "time": "2-4 hours"
        },
        "reactionmetrics": [{
            "scalabilityindex":
            "8",
            "confidenceestimate":
            0.99,
            "closestliterature":
            "Sulfonylation of amines with sulfonyl chlorides"
        }]
    }, {
        "step":
        "2",
        "reactants": [{
            "smiles": "O=S(=O)(Cl)Cc1noc2ccccc12",
            "reactant_metadata": {
                "name": "",
                "chemical_formula": "C8H6ClNO3S",
                "mass": 230.975691732
            }
        }, {
            "smiles": "COc1ccc(CNCc2ccc(OC)cc2)cc1",
            "reactant_metadata": {
                "name": "",
                "chemical_formula": "C16H19NO2",
                "mass": 257.141578848
            }
        }],
        "reagents": [{
            "smiles": "[Na+].[OH-]",
            "product_metadata": {
                "name": "",
                "chemical_formula": "HNaO",
                "mass": 39.99250893200001
            }
        }],
        "products": [{
            "smiles": "COc1ccc(CN(Cc2ccc(OC)cc2)S(=O)(=O)Cc2noc3ccccc23)cc1",
            "product_metadata": {
                "name": "",
                "chemical_formula": "C24H24N2O5S",
                "mass": 452.14059286799994
            }
        }],
        "conditions": {
            "temperature":
            "Room temperature (20-25 \u00b0C)",
            "pressure":
            "Atmospheric pressure",
            "solvent":
            "Polar aprotic solvent such as DMF, DMSO, or acetonitrile",
            "time":
            "2-24 hours, depending on the specific substrates and scale of the reaction"
        },
        "reactionmetrics": [{
            "scalabilityindex":
            "8",
            "confidenceestimate":
            0.86,
            "closestliterature":
            "Sulfonamide formation via nucleophilic substitution of a sulfonyl chloride with a secondary amine"
        }]
    }, {
        "step":
        "3",
        "reactants": [{
            "smiles": "O=P(Cl)(Cl)Cl",
            "reactant_metadata": {
                "name": "",
                "chemical_formula": "Cl3OP",
                "mass": 151.87523428999998
            }
        }, {
            "smiles": "O=S(=O)(O)Cc1noc2ccccc12",
            "reactant_metadata": {
                "name": "",
                "chemical_formula": "C8H7NO4S",
                "mass": 213.009578704
            }
        }],
        "reagents": [{
            "smiles": "O",
            "product_metadata": {
                "name": "",
                "chemical_formula": "H2O",
                "mass": 18.010564684
            }
        }],
        "products": [{
            "smiles": "O=S(=O)(Cl)Cc1noc2ccccc12",
            "product_metadata": {
                "name": "",
                "chemical_formula": "C8H6ClNO3S",
                "mass": 230.975691732
            }
        }],
        "conditions": {
            "temperature": "0-25 \u00b0C",
            "pressure": "1 atm",
            "solvent":
            "Dichloromethane (CH2Cl2) or other non-polar aprotic solvent",
            "time": "1-4 hours"
        },
        "reactionmetrics": [{
            "scalabilityindex":
            "9",
            "confidenceestimate":
            0.93,
            "closestliterature":
            "Chlorination of sulfonic acids using phosphorus oxychloride (POCl3)"
        }]
    }, {
        "step":
        "4",
        "reactants": [{
            "smiles": "COc1ccc(C=O)cc1",
            "reactant_metadata": {
                "name": "",
                "chemical_formula": "C8H8O2",
                "mass": 136.052429496
            }
        }, {
            "smiles": "COc1ccc(CN)cc1",
            "reactant_metadata": {
                "name": "",
                "chemical_formula": "C8H11NO",
                "mass": 137.084063972
            }
        }],
        "reagents": [{
            "smiles": "[H][H]",
            "product_metadata": {
                "name": "",
                "chemical_formula": "H2",
                "mass": 2.015650064
            }
        }, {
            "smiles": "[Na+].[BH4-]",
            "product_metadata": {
                "name": "",
                "chemical_formula": "H4BNa",
                "mass": 38.030374808000005
            }
        }, {
            "smiles": "CC(=O)O.[H][H]",
            "product_metadata": {
                "name": "",
                "chemical_formula": "C2H6O2",
                "mass": 62.036779432
            }
        }],
        "products": [{
            "smiles": "COc1ccc(CNCc2ccc(OC)cc2)cc1",
            "product_metadata": {
                "name": "",
                "chemical_formula": "C16H19NO2",
                "mass": 257.141578848
            }
        }],
        "conditions": {
            "temperature": "Room temperature (20-25\u00b0C)",
            "pressure": "Atmospheric pressure",
            "solvent": "Methanol (MeOH) or Ethanol (EtOH)",
            "time": "1-3 hours"
        },
        "reactionmetrics": [{
            "scalabilityindex":
            "9",
            "confidenceestimate":
            0.88,
            "closestliterature":
            "Reductive amination of an aldehyde with a primary amine using sodium borohydride"
        }]
    }]
}

# Deepseek

In [2]:
from src.utils.llm import call_LLM, llm_pipeline
from src.cache import clear_cache_for_molecule

In [3]:
mol2 = "NC(=O)C1=NC(F)=CN=C1O"

In [4]:
mol = "ClC(C(OC)=C1)=CC2=C1[C@@H]3[C@@H](N(C)CC2)CCC4=CC=CC=C43"

In [5]:
clear_cache_for_molecule(mol)

In [6]:
out = llm_pipeline(mol, LLM="deepinfra/deepseek-ai/DeepSeek-R1")

Calling LLM with molecule: ClC(C(OC)=C1)=CC2=C1[C@@H]3[C@@H](N(C)CC2)CCC4=CC=CC=C43 and run: 0.0
Calling deepinfra/deepseek-ai/DeepSeek-R1 with molecule: ClC(C(OC)=C1)=CC2=C1[C@@H]3[C@@H](N(C)CC2)CCC4=CC=CC=C43
Received response from LLM: <think>
Okay, let's tackle this retrosynthesis problem. The target molecule is ClC(C(OC)=C1)=CC2=C1[C@@H]3[C@@H](N(C)CC2)CCC4=CC=CC=C43. First, I need to analyze its structure to identify possible disconnections.

Looking at the structure, there's a chlorine atom attached to a benzene ring. That makes me think of electrophilic aromatic substitution, maybe using a Friedel-Crafts alkylation or acylation. But wait, the substituents might direct the incoming electrophile. The chlorine is a meta director, so maybe that's a clue.

Then there's the oxygen atom in the OC group. That could be an ester, so perhaps hydrolysis of an ester to a carboxylic acid or vice versa. But the OC is part of a carbonyl group (C(OC)=C), which suggests an ester or ketone. Wait,

[00:07:57] Explicit valence for atom # 9 N, 4, is greater than permitted


In [7]:
out[0]

[['C(C(OC)=C1)=CC2=C1[C@@H]3[C@@H](N(C)CC2)CCC4=CC=CC=C43', 'Cl'],
 ['O=C(C=C1C2=C1[C@@H]3[C@@H](N(C)CC2)CCC4=CC=CC=C43)C', 'CO']]

In [1]:
# Assume openai>=1.0.0
from openai import OpenAI

# Create an OpenAI client with your deepinfra token and endpoint
openai = OpenAI(
    api_key="wFYChdhelLBf5R49oWGLyKhAbqWsBgMn",
    base_url="https://api.deepinfra.com/v1/openai",
)

chat_completion = openai.chat.completions.create(
    model="deepseek-ai/DeepSeek-R1",
    messages=[{
        "role": "user",
        "content": "Hello"
    }],
)

print(chat_completion.choices[0].message.content)
print(chat_completion.usage.prompt_tokens,
      chat_completion.usage.completion_tokens)

# Hello! It's nice to meet you. Is there something I can help you with, or would you like to chat?
# 11 25


<think>

</think>

Hello! How can I assist you today? 😊
4 16
